In [1]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    override fun toString(): String = "$name: $score ± $error ns/op"
}

val hotspotBenchmarkLines = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("JmhBenchmark\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
    }
hotspotBenchmarkLines.joinToString("\n")

baselineDouble: 941916.132 ± 4429.574 ns/op
baselineFloat: 910182.882 ± 3525.255 ns/op
inlineDouble: 950208.459 ± 19364.968 ns/op
inlineFloat: 914927.291 ± 6581.754 ns/op
longPackFloat: 1589400.572 ± 17887.677 ns/op
mutableRefDouble: 649870.736 ± 5998.91 ns/op
mutableRefFloatIn1Long: 1566596.328 ± 7695.366 ns/op
mutableRefFloatIn2Longs: 657675.499 ± 10049.779 ns/op
valueDouble: 679555.47 ± 3147.92 ns/op
valueFloat: 666612.502 ± 8971.526 ns/op
valueInlineDouble: 657662.914 ± 7954.246 ns/op
valueInlineFloat: 673786.95 ± 9490.854 ns/op

In [2]:
%use lets-plot

In [3]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(platform: String, benchmarkLines: List<BenchmarkData>) {
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = listOf("Float", "Double").associateWith { type ->
        benchmarkLines.single { it.name == "baseline$type" }
    }
    val plot = letsPlot() +
            labs("Microbenchmark of 3D scene rendering ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 0.5),
                size = 5,
            ) {
                x = names
                y = scores
                label = benchmarkLines.map {
                    val isFloat = it.name.contains("Float")
                    val type = if (isFloat) "Float" else "Double"
                    val score = it.score / baseline[type]!!.score
                    val error = it.error / baseline[type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to scoresPlusErrors.max(), name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$platform.svg")
    plot.show()
}


In [4]:
makePlot("Hotspot", hotspotBenchmarkLines)

In [5]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

val artBenchmarkLines = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} \\d+:\\d+:\\d+.\\d+ \\d+ \\d+ I Benchmark: AndroidBenchmark.([^\\[]+)\\[Metric \\(timeNs\\) results: median ([^,]+), min [^,]+, max [^,]+, standardDeviation: ([^,]+), .*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = hotspotBenchmarkLines.mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         lines.sortedBy { indexes[it.name]!! }
     }
artBenchmarkLines.joinToString("\n")

baselineDouble: 1.0222239277777778E7 ± 400030.9104534168 ns/op
baselineFloat: 1.1463365375E7 ± 401479.2494035714 ns/op
inlineDouble: 2.3791822125E7 ± 875003.7124902818 ns/op
inlineFloat: 1.8160392299999997E7 ± 862164.0314703624 ns/op
longPackFloat: 4500319.068181818 ± 15783.62096242716 ns/op
mutableRefDouble: 3191629.8125 ± 57969.18846950532 ns/op
mutableRefFloatIn1Long: 5753390.375 ± 13898.516651497792 ns/op
mutableRefFloatIn2Longs: 2723149.423076923 ± 51419.33878301706 ns/op
valueDouble: 1.19470865E7 ± 508585.7098869036 ns/op
valueFloat: 7555263.307692308 ± 351407.9795457559 ns/op
valueInlineDouble: 497368.9797979798 ± 10144.918485427219 ns/op
valueInlineFloat: 480918.3381642512 ± 12565.344512456217 ns/op

In [6]:
makePlot("ART", artBenchmarkLines)